In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
from scipy import stats
import scanpy as sc
import xlsxwriter

In [2]:
#Import dataset1 for either training or testing
global_dir_path = "C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data"
#4wks
# dir_path = global_dir_path + '/2023_0809_0810_Breath_Eml4Alk_4wks_multiplex/Spectra/Peaks/Int_ratio_r1'

#5wks
# dir_path = global_dir_path + '/2023_0816_0817_Breath_Eml4Alk_5wks_multiplex/All samples_spectra/Peaks/Int_ratio_r1'

#6wks
# dir_path = global_dir_path + '/2023_May_June_Eml4Alk_6to8wks_spectra/2023_05_19_Breath_Eml4Alk_6wks_calibrated_spectra/Peaks/Int_ratio_r1'

#6wks August
# dir_path = global_dir_path + '/2023_08_24_Breath_EmlAlk_6wks_multiplex/Spectra/Peaks/Int_ratio_r1'

#4wks October
# dir_path = global_dir_path + '/2023_10_19_Breath_Eml4Alk_4wk_5plex/Spectra/Peaks/Int_ratio_r1'

#5wks October
# dir_path = global_dir_path + '/2023_10_26_Breath_Eml4Alk_5wks_cohort2/spectra/Peaks/Int_ratio_r1'

#6wks October failed longitudinal - not used
# dir_path = global_dir_path + '/2023_10_23_Breath_Eml4-Alk_6wks_cohort1/spectra/Peaks/Int_ratio_r1'

#6wks Jan2024
# dir_path = global_dir_path + '/2024_0116_0117_Breath_Eml4aLK_6w_endo/Peaks/Int_ratio_r1'

#5ws Feb2024
# dir_path = global_dir_path + '/2024_02_09_Eml4-Alk_5wks_multiplex/Spectra/Peaks/Int_ratio_r1'

#6wks Feb2024
dir_path = global_dir_path + '/2024_02_16_Eml4Alk_6wks_multiplex/spectra/Peaks/Int_ratio_r1'


dir_list = sorted(os.listdir(dir_path))
print(dir_list)

# dir_list2 = sorted(os.listdir(dir_path2))
# print(dir_list2)

#for intensity, remember to change the df1 column title from auc ratio to intensity ratio


['2024_02_16_W6_EA4_1_peaks.xlsx', '2024_02_16_W6_EA4_2_peaks.xlsx', '2024_02_16_W6_EA4_3_peaks.xlsx', '2024_02_16_W6_EA4_4_peaks.xlsx', '2024_02_16_W6_EA5_1_peaks.xlsx', '2024_02_16_W6_EA5_2_peaks.xlsx', '2024_02_16_W6_EA5_3_peaks.xlsx', '2024_02_16_W6_EA5_4_peaks.xlsx', '2024_02_16_W6_EA5_5_peaks.xlsx', '2024_02_16_W6_Healthy2_1_peaks.xlsx', '2024_02_16_W6_Healthy2_2_peaks.xlsx', '2024_02_16_W6_Healthy2_3_peaks.xlsx', '2024_02_16_W6_Healthy2_4_peaks.xlsx', '2024_02_16_W6_Healthy2_5_peaks.xlsx']


In [3]:
df_dict = {}
for i in range(len(dir_list)):
    name = dir_path + '/' + dir_list[i]
    with open(name, mode="r", encoding="iso-8859-1") as file:
        df = pd.read_excel(name, index_col=0)
        
        df_dict[str(i)] = df

In [4]:
workbook = dir_path + '/matrix.xlsx'
writer = pd.ExcelWriter(workbook, engine = 'xlsxwriter')

# Write each DataFrame to a separate sheet in the Excel file
for sheet_name, df in df_dict.items():
    df.to_excel(writer, sheet_name=sheet_name, index=False)

writer.close()

In [5]:
xl = pd.ExcelFile(workbook)
sheets = xl.sheet_names
df1 = pd.concat(pd.read_excel(workbook, sheet_name=s).assign(Sample_ID = s) for s in sheets)

In [6]:
df1.head(800)

,m/z,Intensity_ratio,Intensity,Sample_ID
0,26.9,0.001690,15.310144,0
1,31.0,0.009590,86.880890,0
2,33.0,0.424769,3848.053228,0
3,33.1,0.002117,19.175683,0
4,34.0,0.006004,54.387020,0
...,...,...,...,...
57,91.0,0.014166,131.588054,4
58,91.1,0.010693,99.329951,4
59,92.0,0.001618,15.030560,4
60,92.1,0.002053,19.067630,4


In [7]:
#Not sure - previously thought about picking out the peaks that are in cancer but not in healthy

In [8]:
masses = [df1['m/z'].iloc[i] for i in range(len(df1))]
# print(masses)

In [9]:
#set initial values of each mass as zero
d = {}
num_samples = len(dir_list) 
for mass in masses:
    d[mass] = [0]*num_samples

In [10]:
#add the actual values for samples with that m/z; if sample did not have that m/z, keep it zero
for i in range(len(df1)):
    mass = df1['m/z'].iloc[i]
#     auc = df1['AUC_ratio'].iloc[i]
    intensity = df1['Intensity_ratio'].iloc[i] #use this for intensity ratio
#     print(type(auc))
    index = df1['Sample_ID'].iloc[i]
#     print(int(index))
#     d[mass][int(index)] = auc
    d[mass][int(index)] = intensity

    
#print(d)

In [11]:
df2 = pd.DataFrame(data=d)
df2
# df2

,26.9,31.0,33.0,33.1,34.0,38.0,38.9,40.0,40.9,41.0,...,138.9,146.9,147.9,174.9,223.2,138.2,84.0,113.2,179.2,177.2
0,0.001690,0.009590,0.424769,0.002117,0.006004,0.003342,0.001894,0.003547,0.003288,0.491155,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.008513,0.599447,0.000000,0.007519,0.002024,0.000000,0.002373,0.002025,0.321986,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.008833,0.824924,0.000000,0.010567,0.002241,0.001412,0.002239,0.003204,0.318413,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.008805,0.625289,0.000000,0.007530,0.002590,0.001347,0.002226,0.000000,0.294864,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.008310,0.259294,0.004109,0.003676,0.004067,0.000000,0.001836,0.002031,0.336585,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.009391,1.018593,0.000000,0.012100,0.003167,0.000000,0.002235,0.002399,0.343140,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.010495,0.872657,0.000000,0.011375,0.002504,0.001849,0.002033,0.000000,0.297802,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.011120,0.882787,0.000000,0.010821,0.004550,0.001852,0.003099,0.003260,0.354351,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.011186,0.582403,0.000000,0.007299,0.003764,0.002398,0.003274,0.002983,0.392719,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.011712,0.925630,0.000000,0.010374,0.003044,0.000000,0.002797,0.003319,0.376767,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
df2 = df2[sorted(df2.columns)]
df2

,26.9,31.0,33.0,33.1,34.0,35.0,38.0,38.9,40.0,40.9,...,222.9,223.0,223.1,223.2,224.0,224.1,224.9,225.0,225.1,237.0
0,0.001690,0.009590,0.424769,0.002117,0.006004,0.000000,0.003342,0.001894,0.003547,0.003288,...,0.000000,0.010143,0.000000,0.000000,0.000000,0.000000,0.000000,0.005124,0.000000,0.000000
1,0.000000,0.008513,0.599447,0.000000,0.007519,0.002107,0.002024,0.000000,0.002373,0.002025,...,0.000000,0.007584,0.006997,0.000000,0.000000,0.000000,0.000000,0.003791,0.000000,0.000000
2,0.000000,0.008833,0.824924,0.000000,0.010567,0.002686,0.002241,0.001412,0.002239,0.003204,...,0.000000,0.007189,0.000000,0.000000,0.000000,0.000000,0.000000,0.003149,0.000000,0.000000
3,0.000000,0.008805,0.625289,0.000000,0.007530,0.002498,0.002590,0.001347,0.002226,0.000000,...,0.000000,0.009381,0.000000,0.000000,0.000000,0.000000,0.000000,0.004122,0.000000,0.000000
4,0.000000,0.008310,0.259294,0.004109,0.003676,0.000000,0.004067,0.000000,0.001836,0.002031,...,0.000000,0.012473,0.008867,0.000000,0.000000,0.000000,0.000000,0.006173,0.005545,0.001618
5,0.000000,0.009391,1.018593,0.000000,0.012100,0.003152,0.003167,0.000000,0.002235,0.002399,...,0.000000,0.009192,0.005728,0.000000,0.000000,0.000000,0.000000,0.000000,0.004514,0.000000
6,0.000000,0.010495,0.872657,0.000000,0.011375,0.003049,0.002504,0.001849,0.002033,0.000000,...,0.000000,0.008720,0.003596,0.000000,0.000000,0.001386,0.001568,0.004734,0.000000,0.000000
7,0.000000,0.011120,0.882787,0.000000,0.010821,0.002979,0.004550,0.001852,0.003099,0.003260,...,0.003218,0.009043,0.007288,0.000000,0.002024,0.000000,0.000000,0.004674,0.000000,0.000000
8,0.000000,0.011186,0.582403,0.000000,0.007299,0.002180,0.003764,0.002398,0.003274,0.002983,...,0.000000,0.008498,0.007044,0.000000,0.000000,0.000000,0.000000,0.005232,0.000000,0.000000
9,0.000000,0.011712,0.925630,0.000000,0.010374,0.002669,0.003044,0.000000,0.002797,0.003319,...,0.000000,0.009444,0.008767,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
df2.insert(0, "Sample_ID", dir_list)

In [14]:
types = ['Healthy' if '_C' in i or '_H' in i else 'Treated' if '_T' in i else 'Cancer' for i in df2['Sample_ID']]
df2.insert(0, 'Sample_type', types)
df2

,Sample_type,Sample_ID,26.9,31.0,33.0,33.1,34.0,35.0,38.0,38.9,...,222.9,223.0,223.1,223.2,224.0,224.1,224.9,225.0,225.1,237.0
0,Cancer,2024_02_16_W6_EA4_1_peaks.xlsx,0.001690,0.009590,0.424769,0.002117,0.006004,0.000000,0.003342,0.001894,...,0.000000,0.010143,0.000000,0.000000,0.000000,0.000000,0.000000,0.005124,0.000000,0.000000
1,Cancer,2024_02_16_W6_EA4_2_peaks.xlsx,0.000000,0.008513,0.599447,0.000000,0.007519,0.002107,0.002024,0.000000,...,0.000000,0.007584,0.006997,0.000000,0.000000,0.000000,0.000000,0.003791,0.000000,0.000000
2,Cancer,2024_02_16_W6_EA4_3_peaks.xlsx,0.000000,0.008833,0.824924,0.000000,0.010567,0.002686,0.002241,0.001412,...,0.000000,0.007189,0.000000,0.000000,0.000000,0.000000,0.000000,0.003149,0.000000,0.000000
3,Cancer,2024_02_16_W6_EA4_4_peaks.xlsx,0.000000,0.008805,0.625289,0.000000,0.007530,0.002498,0.002590,0.001347,...,0.000000,0.009381,0.000000,0.000000,0.000000,0.000000,0.000000,0.004122,0.000000,0.000000
4,Cancer,2024_02_16_W6_EA5_1_peaks.xlsx,0.000000,0.008310,0.259294,0.004109,0.003676,0.000000,0.004067,0.000000,...,0.000000,0.012473,0.008867,0.000000,0.000000,0.000000,0.000000,0.006173,0.005545,0.001618
5,Cancer,2024_02_16_W6_EA5_2_peaks.xlsx,0.000000,0.009391,1.018593,0.000000,0.012100,0.003152,0.003167,0.000000,...,0.000000,0.009192,0.005728,0.000000,0.000000,0.000000,0.000000,0.000000,0.004514,0.000000
6,Cancer,2024_02_16_W6_EA5_3_peaks.xlsx,0.000000,0.010495,0.872657,0.000000,0.011375,0.003049,0.002504,0.001849,...,0.000000,0.008720,0.003596,0.000000,0.000000,0.001386,0.001568,0.004734,0.000000,0.000000
7,Cancer,2024_02_16_W6_EA5_4_peaks.xlsx,0.000000,0.011120,0.882787,0.000000,0.010821,0.002979,0.004550,0.001852,...,0.003218,0.009043,0.007288,0.000000,0.002024,0.000000,0.000000,0.004674,0.000000,0.000000
8,Cancer,2024_02_16_W6_EA5_5_peaks.xlsx,0.000000,0.011186,0.582403,0.000000,0.007299,0.002180,0.003764,0.002398,...,0.000000,0.008498,0.007044,0.000000,0.000000,0.000000,0.000000,0.005232,0.000000,0.000000
9,Healthy,2024_02_16_W6_Healthy2_1_peaks.xlsx,0.000000,0.011712,0.925630,0.000000,0.010374,0.002669,0.003044,0.000000,...,0.000000,0.009444,0.008767,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
df2.to_excel(dir_path + '/dataframe.xlsx')

In [16]:
#perform zscore for each column
d2 = {}
num_samples = len(df2)
for col in df2.columns:
    d2[col] = [0]*num_samples

for col in df2.columns: 
    if type(col) == int or type(col) == float: 
        val = []
        for i in range(len(df2)):
            val.append(df2[col].loc[df2.index[i]])
        
        arr = np.array(val)
        score = stats.zscore(arr)
                
        for i in range(len(score)):
            d2[col][i] = score[i]

df3 = pd.DataFrame(data=d2)
df3 = df3.drop(['Sample_ID'], axis=1)
df3['Sample_type'] = types
df3.to_excel(dir_path + '/zscore.xlsx')